# 8/9 Hackathon Project

Here, I perform data cleaning on the covid_19_data.csv data set

Over half of the data is missing province/state information

Here, I drop any data points without a province/state provided because the question asks to group by province/state.

This drops half of the data points.

I think there is a way to so this on AWS GLUE studio but this is my workaround

In [24]:
import pandas as pd

In [32]:
d = pd.read_csv('covid_19_data.csv')

In [33]:
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20574 entries, 0 to 20573
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   SNo              20574 non-null  int64  
 1   ObservationDate  20574 non-null  object 
 2   Province/State   10026 non-null  object 
 3   Country/Region   20574 non-null  object 
 4   Last Update      20574 non-null  object 
 5   Confirmed        20574 non-null  float64
 6   Deaths           20574 non-null  float64
 7   Recovered        20574 non-null  float64
dtypes: float64(3), int64(1), object(4)
memory usage: 1.3+ MB


In [34]:
data = d.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10026 entries, 0 to 20573
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   SNo              10026 non-null  int64  
 1   ObservationDate  10026 non-null  object 
 2   Province/State   10026 non-null  object 
 3   Country/Region   10026 non-null  object 
 4   Last Update      10026 non-null  object 
 5   Confirmed        10026 non-null  float64
 6   Deaths           10026 non-null  float64
 7   Recovered        10026 non-null  float64
dtypes: float64(3), int64(1), object(4)
memory usage: 705.0+ KB


In [35]:
data.to_csv('covid_19_data2.csv', index=False)

In [36]:
data = pd.read_csv('covid_19_data2.csv')

In [39]:
data.head()

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0


In [ ]:
# This is the script that I generated on AWS Glue Studio 

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.dynamicframe import DynamicFrame
from pyspark.sql import functions as SqlFuncs


def sparkAggregate(
    glueContext, parentFrame, groups, aggs, transformation_ctx
) -> DynamicFrame:
    aggsFuncs = []
    for column, func in aggs:
        aggsFuncs.append(getattr(SqlFuncs, func)(column))
    result = (
        parentFrame.toDF().groupBy(*groups).agg(*aggsFuncs)
        if len(groups) > 0
        else parentFrame.toDF().agg(*aggsFuncs)
    )
    return DynamicFrame.fromDF(result, glueContext, transformation_ctx)


args = getResolvedOptions(sys.argv, ["JOB_NAME"])
sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args["JOB_NAME"], args)

# Script generated for node AWS Glue Data Catalog
AWSGlueDataCatalog_node1660058383506 = glueContext.create_dynamic_frame.from_catalog(
    database="hackathon-db",
    table_name="hackathon_bucket_17",
    transformation_ctx="AWSGlueDataCatalog_node1660058383506",
)

# Script generated for node Aggregate
Aggregate_node1660058393535 = sparkAggregate(
    glueContext,
    parentFrame=AWSGlueDataCatalog_node1660058383506,
    groups=["province/state", "country/region"],
    aggs=[["confirmed", "max"], ["deaths", "max"], ["recovered", "max"]],
    transformation_ctx="Aggregate_node1660058393535",
)

# Script generated for node Amazon S3
AmazonS3_node1660058429891 = glueContext.write_dynamic_frame.from_options(
    frame=Aggregate_node1660058393535,
    connection_type="s3",
    format="glueparquet",
    connection_options={"path": "s3://hackathon-bucket-17/", "partitionKeys": []},
    format_options={"compression": "snappy"},
    transformation_ctx="AmazonS3_node1660058429891",
)

job.commit()

In [28]:
import pyarrow.parquet as pq #pq to read parquet files

df = pq.read_table('run-AmazonS3_node1660058429891-2-part-block-0-r-00003-snappy.parquet')
df.to_pandas()

,province/state,country/region,max_confirmed_#0,max_deaths_#1,max_recovered_#2
0,Heilongjiang,Mainland China,944.0,13.0,605.0
1,Henan,Mainland China,1276.0,22.0,1254.0
2,Hunan,Mainland China,1019.0,4.0,1015.0
3,Jiangsu,Mainland China,653.0,0.0,650.0
4,Ningxia,Mainland China,75.0,0.0,75.0
...,...,...,...,...,...
58,US,US,1.0,0.0,171.0
59,Greenland,Denmark,11.0,0.0,11.0
60,Jervis Bay Territory,Australia,0.0,0.0,0.0
61,Yukon,Canada,11.0,0.0,0.0
